In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Reshape,Activation,Attention,MaxPool1D,Dense, Conv1D, Lambda,Convolution2D, GRU, LSTM, Bidirectional, TimeDistributed,
                          Dropout, Flatten, LayerNormalization,RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import load_model
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import linear_model
import pandas as pd
import torch
import torch.nn as nn
import random
# from keras import layers as layers
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from scipy import special
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
#from keras_self_attention import SeqSelfAttention

from sklearn.preprocessing import StandardScaler
import os
import random
import numpy as np
import json
import matplotlib.pyplot
import pickle
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.manifold import TSNE

In [2]:
def Identity_Block(X, time_step, kernel_size, drop_out ):
    
    x = Conv1D(time_step, kernel_size, padding='same')(X)
    x = LayerNormalization(axis=1)(x)
    x = Activation('relu')(x)
    x = tf.transpose(x, (0, 2, 1)) 
    x = Attention()([x,x])
    x = tf.transpose(x, (0, 2, 1))
    x = Dropout(drop_out)(x)
    
    return x

In [3]:
def Conv1D_Block(X, time_step, kernel_size, drop_out):
    X_input = X
    x = Conv1D(time_step, kernel_size, padding='same')(X)
    x = LayerNormalization(axis=1)(x)
    x = Activation('relu')(x)
    x = tf.transpose(x, (0, 2, 1)) 
    x = Attention()([x,x])
    x = tf.transpose(x, (0, 2, 1)) 
    
    x = Conv1D(time_step, kernel_size, padding='same')(x)
    x = LayerNormalization(axis=1)(x)
    x = Activation('relu')(x)
    x = tf.transpose(x, (0, 2, 1)) 
    x = Attention()([x,x])
    x = tf.transpose(x, (0, 2, 1)) 
    
    x = Conv1D(time_step, kernel_size, padding='same')(x)
    x = LayerNormalization(axis=1)(x)
    x = tf.transpose(x, (0, 2, 1)) 
    x = Attention()([x,x])
    x = tf.transpose(x, (0, 2, 1))
    x = Dropout(drop_out)(x)
    
    x = layers.add([X_input,x])
    x = Activation('relu')(x)
    
    return x

In [4]:
def Conv1D_Pie(size = 12, time_step = 128, kernel_size = 3, lr = 2e-5, embedding = 8, 
                  n_step = 96, drop_out=0.2, l2_norm = 5e-7):
    X_input = layers.Input(shape=(n_step,1))
    X = X_input
#    Embedding_out = layers.Input(shape=(8,))
    
    x = Identity_Block(X, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = MaxPooling1D(2, padding='same')(x)
    x = Identity_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = MaxPooling1D(2, padding='same')(x)
# x = Conv1D(128, 3, activation='relu', padding='same')(x)
# encoded = MaxPooling1D(2, padding='same')(x)
    x = tf.transpose(x, (0, 2, 1)) 
    encoded = Bidirectional(LSTM(size, return_sequences=False, dropout=drop_out))(x)
# # at this point the representation is (4, 4, 8) i.e. 128-dimensional
    embedding = Dense(embedding, activation='relu',activity_regularizer=l2(l2_norm))(encoded)
    
    repeat = RepeatVector(128)(embedding)
    x = tf.transpose(repeat, (0, 2, 1)) 
    
    x = Identity_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = UpSampling1D(2)(x)
    x = Identity_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = UpSampling1D(2)(x)
    x = Identity_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = UpSampling1D(3)(x)
    x = Identity_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    x = Conv1D_Block(x, time_step, kernel_size, drop_out)
    decoded = Conv1D(1, 3, activation='linear', padding='same')(x)
#x = UpSampling1D(2)(x)

    model = Model(X_input, decoded,name = 'Convolutional_1D_with_Attention')
    model.compile(Adam(lr), loss='mse')
    
    return model